<a href="https://colab.research.google.com/github/IshtiSikder/Optimal-allocation-of-rescue-teams-for-hostage-situations/blob/test/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CTMC MDP model for rescue operational system!!!

prob <- function(opSet = policySet, t, lam, eps, mu) {
  opTRM = matrix(0, nrow = t+1, ncol = t+1)
  for (i in 2:t) {
    opTRM[i, i-1] = sum(i*eps, opSet[i]*mu)
    opTRM[i, i+1] = lam
  }
  opTRM[1, 2] = lam
  opTRM[t+1, t] = sum(t*eps, opSet[t+1]*mu)

  k = t+1
  steady_prob <- matrix(0, nrow=1, ncol=k)
  state <- matrix(0, nrow=1, ncol=k-1)
  for (i in 1:(k-1)) {
    if (i-2 < 0) {
      state[i] <- (lam/opTRM[i+1,i])
    } else {
      state[i] <- state[i-1]*(lam/opTRM[i+1,i])
    }
  }

  steady_prob[1] <- (1+sum(state))^-1
  for (i in 2:k) {
    steady_prob[i] <- state[i-1]*steady_prob[1]
  }
  return(steady_prob)
}




# Check function to determine new policy is same with current or not
checkPolicy = function(policySet, policySetnew) {
  n = 0
  for (i in policySet == policySetnew) {
    if (i == 'FALSE') {
      n = 1
    }
  }
  if (n == 0) {
    return("TRUE")
  } else {
    return("FALSE")
  }
}


# Define MDP function (for basic policy)
MDP = function(lam, mu, eps, numRescue, numTerror, costT, RC, mult) {
  r = numRescue
  t = numTerror

  #calculating costR

  costR = mult*costT*(mu+eps)


  #########################################################
  #EDITS: 22ND FEB,2023
  #NEW SERVICE RATES STRUCTURE, ADD ORIGINAL ONE AS WELL
  aijMatrix = array(0, dim = c(t+1,t+1,r+1))
  for (k in 1:(r+1)) {
    for (i in 1:t) {
      aijMatrix[i,i+1,k] <- lam
      if (i > 1) {
        #if (k==4){
          #if(i==4){
            #cat('min(k-1,i-1)',min(k-1,i-1))}}
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)
      }
    }
    aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
    aijMatrix[1,2,k] <- lam
}

  for (k in 1:(r+1)) {
    for (i in 1:(t+1)) {

      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])
    }
  }
  #########################################################


  # Design the MDP algorithm
  # Value determination
  policy = array(0, dim = c(1,t+1))
  policyOld = array(0, dim = c(1,t+1))
  policyNew = array(1, dim=c(1,t+1)) # initially set all decision is 1 in first policy
  policyTrack = array(2, dim=c(1,t+1))
  iteration = 0
  gainMatrix = c()

  while (checkPolicy(policyOld, policyNew) == 'FALSE') {
    policyOld = policyNew
    policy = policyNew
    # Design the transition matrix regarding policy
    A = array(0, dim = c(t+1,t+1))
    for (i in 1:length(policy)) {
      A[i,] = aijMatrix[i,,(policy[i])]
    }

    # Design the cost matrix regarding policy

    q = array(0,dim=c(t+1,1))
    for (i in 1:length(policy)) {

      if (RC == 'L'){
      cost = (policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost = sqrt(policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost = ((policy[i]-1)^2)*costR + (i-1)*eps*costT
      }

      q[i,1] = cost
    }


    newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
    solution = solve(newA) %*% q
    gain = solution[1] # First row of solution is gain value
    gainMatrix = append(gainMatrix, gain)

    # Policy improvement
    newsolution = array(0, dim=c(t+1,1))
    for (i in 1:t) {
      newsolution[i] = solution[i+1]
    }


    newsolution[t+1] = 0 # Set last v value as 0 to calcultate the test quantity in policy improvement
    improveMatrix = array(0, dim=c(t+1,r+1))
    cost = array(0,dim=c(t+1,r+1))

    for (i in 1:(t+1)) {
      for (j in 1:(r+1)) {
        # Cost when we have j rescue operation and i terrorists

        if (RC == 'L'){
      cost[i,j] = (j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost[i,j] = sqrt(j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost[i,j] = ((j-1)^2)*costR + (i-1)*eps*costT
      }

        improveMatrix[i,j] = cost[i,j] + aijMatrix[i,,j] %*% newsolution

      }
    }

    # pick the minimum value among the decision
    policyNew = array(1, dim = c(1,t+1))
    for (i in 1:length(policyNew)) {
      policyNew[i] = which.min(improveMatrix[i,])
    }
    iteration = iteration + 1
    policyTrack = rbind(policyTrack, policyNew)
    policyNew
  }
  for (i in 1:length(policyNew)) {
    policyNew[i] = policyNew[i] - 1
  }
  for (i in 1:nrow(policyTrack)) {
    for (j in 1:ncol(policyTrack)) {
      policyTrack[i,j] = policyTrack[i,j] - 1
    }

  steady = prob(opSet = policyNew, t=numTerror, lam = lam, eps = eps, mu = mu)

  gain = solution[1]

  }


   #return(list(policyNew,improveMatrix,cost,aijMatrix,newsolution))

   return (policyNew)

 }

In [2]:
fixed = function(policy,mult,rc,costT,lam,mu,eps,t,r){ #Let i represent no. of hostage incidents, and k represent no. of rescue teams

  policy = policy + 1

  aijMatrix = array(0, dim = c(t+1,t+1,r+1)) #build a 3d zero transitional matrix with t+1 rows, t+1 cols, and r+1 depth
  for (k in 1:(r+1)) {
    for (i in 1:t) {
      aijMatrix[i,i+1,k] <- lam               #for every transition from i to i+1 at any layer of depth, arrival rate lam remains the same
      if (i > 1) {
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)  #for every transition from i to i-1 (i>0) at any layer of depth,
                                                              #service rate is denoted by (min(k,i)*mu)+(i*eps)
      }
    }
    aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
    aijMatrix[1,2,k] <- lam
}

  for (k in 1:(r+1)) {
    for (i in 1:(t+1)) {

      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k]) #Each row for all layers of depth sums upto 0.
    }
  }


#So arrival rate of hostage incidents is not impacted by number of available rescue teams.
#The service rate to go back one hostage incident less in the system is dependent both upon
#the number of rescue teams available and current number of hostage incidents.
#The 3d matrix contains all these arrival and departure rates over 10 hostage incidents and 10 rescue teams.


  A = array(0, dim = c(t+1,t+1))

  for (i in 1:length(policy)) {

      A[i,] = aijMatrix[i,,(policy[i])]
    }

    # Design the cost matrix regarding policy

    q = array(0,dim=c(t+1,1))

    costR = mult*costT*(mu+eps)

    for (i in 1:length(policy)) {

      if (rc == 'L'){
      cost = (policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (rc == 'SR'){
      cost = sqrt(policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (rc == 'Q'){
      cost = ((policy[i]-1)^2)*costR + (i-1)*eps*costT
      }

      q[i,1] = cost
    }


    newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
    solution = solve(newA) %*% q
    gain = solution[1]

    return(gain)


}











mult_test = function(cost_terror=1,rc,t=10,r=10,lam=0.2763,mu=0.2829,eps=0.1718,low_lim,up_lim,incr,policy=c(0,1,2,3,4,5,6,7,8,9,10)) {

prev_result = array(100,dim=c(1,t+1))


list_mult = list()
list_sum = list()
list_gain = list()
list_fixed = list()
list_policy = list()


aijMatrix = array(0, dim = c(t+1,t+1,r+1))
  for (k in 1:(r+1)) {
    for (i in 1:t) {
      aijMatrix[i,i+1,k] <- lam
      if (i > 1) {
        #if (k==4){
          #if(i==4){
            #cat('min(k-1,i-1)',min(k-1,i-1))}}
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)
      }
    }
    aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
    aijMatrix[1,2,k] <- lam
}

  for (k in 1:(r+1)) {
    for (i in 1:(t+1)) {

      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])
    }
  }

if (low_lim != up_lim){

for (val in seq(low_lim,up_lim, by = incr)){

        current_result = MDP(lam, mu, eps, numRescue=r, numTerror=t, costT=cost_terror, RC=rc, mult=val)


        if (!(identical(current_result,prev_result))) {

          list_policy = append(list_policy,list(current_result))

          steady_prob = prob(current_result, t, lam, mu, eps)

          sum = 0

          for (i in 1:length(current_result)){
            sum = sum + current_result[i]*steady_prob[i]
          }

          sum = sum*mu

          list_mult = append(list_mult,val)
          list_sum = append(list_sum,sum)

          current = current_result + 1

          A = array(0, dim = c(11,11))
          for (i in 1:length(current)) {
          A[i,] = aijMatrix[i,,(current[i])]
          }

          # Design the cost matrix regarding policy

          q = array(0,dim=c(11,1))

          costT = cost_terror

          costR = val*costT*(mu+eps)

          for (i in 1:length(current)) {

          if (rc == 'L'){
          cost = (current[i]-1)*costR + (i-1)*eps*costT
          }

          if (rc == 'SR'){
          cost = sqrt(current[i]-1)*costR + (i-1)*eps*costT
          }

          if (rc == 'Q'){
          cost = ((current[i]-1)^2)*costR + (i-1)*eps*costT
          }

          q[i,1] = cost
          }


          newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
          solution = solve(newA) %*% q
          gain = solution[1]

          fixed_gain = fixed(policy,val,rc,cost_terror,lam,mu,eps,t,r)

          list_gain = append(list_gain,gain)
          list_fixed = append(list_fixed,fixed_gain)



          cat('\n\n','mult:')
          print(val)
          cat('\n','Rate of success/failure:')
          print(sum)
          cat('\n','gain:')
          print(gain)

          cat('\n','gain for fixed policy:')
          print(fixed_gain)

          cat('\n','change in gain compared to fixed policy:')
          print(gain-fixed_gain)


          cat('\nRC:',rc,',','\n\n')
          cat('optimal policy:\n\n')
          print(current_result)
          cat('\nsteady_prob:\n')
          print(steady_prob)



          prev_result = current_result

        }

    }

}

else {

  current_result = MDP(lam, mu, eps, numRescue=r, numTerror=t, costT=cost_terror, RC=rc, mult=low_lim)

  list_policy = append(list_policy,list(current_result))

  steady_prob = prob(current_result, t, lam, mu, eps)

  sum = 0

  for (i in 1:length(current_result)){
            sum = sum + current_result[i]*steady_prob[i]
          }

  sum = sum*mu

  list_mult = append(list_mult,low_lim)
  list_sum = append(list_sum,sum)

  current = current_result + 1

  A = array(0, dim = c(11,11))
  for (i in 1:length(current)) {
          A[i,] = aijMatrix[i,,(current[i])]
          }

  # Design the cost matrix regarding policy

  q = array(0,dim=c(11,1))

  costT = cost_terror

  costR = low_lim*costT*(mu+eps)

  for (i in 1:length(current)) {

  if (rc == 'L'){
          cost = (current[i]-1)*costR + (i-1)*eps*costT
          }

  if (rc == 'SR'){
          cost = sqrt(current[i]-1)*costR + (i-1)*eps*costT
          }

  if (rc == 'Q'){
          cost = ((current[i]-1)^2)*costR + (i-1)*eps*costT
          }

  q[i,1] = cost
          }


          newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
          solution = solve(newA) %*% q
          gain = solution[1]

          fixed_gain = fixed(policy,low_lim,rc,cost_terror,lam,mu,eps,t,r)

          list_gain = append(list_gain,gain)
          list_fixed = append(list_fixed,fixed_gain)



          cat('\n\n','mult:')
          print(low_lim)
          cat('\n','Rate of success/failure:')
          print(sum)
          cat('\n','gain:')
          print(gain)

          cat('\n','gain for fixed policy:')
          print(fixed_gain)

          cat('\n','change in gain compared to fixed policy:')
          print(gain-fixed_gain)


          cat('\nRC:',rc,',','\n\n')
          cat('optimal policy:\n\n')
          print(current_result)
          cat('\nsteady_prob:\n')
          print(steady_prob)



          #prev_result = current_result


}

return (list(list_mult,list_sum,list_gain,list_fixed,list_policy))

}

In [3]:
lst = mult_test(cost_terror=1,rc='L',t=10,r=10,lam=0.2763,mu=0.2829,eps=0.1718,low_lim=0,up_lim=3,incr=0.01,policy=c(0,1,2,3,4,5,6,7,8,9,10))



 mult:[1] 0

 Rate of success/failure:[1] 0.1149728

 gain:[1] 0.1043949

 gain for fixed policy:[1] 0.1043949

 change in gain compared to fixed policy:[1] 0

RC: L , 

optimal policy:

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]    0    1    2    3    4    5    6    7    8     9    10

steady_prob:
          [,1]     [,2]       [,3]        [,4]        [,5]         [,6]
[1,] 0.6765419 0.253428 0.05872865 0.009852292 0.001295232 0.0001399908
             [,7]         [,8]         [,9]        [,10]        [,11]
[1,] 1.284562e-05 1.024077e-06 7.217255e-08 4.557797e-09 2.769561e-10


 mult:[1] 0.54

 Rate of success/failure:[1] 0.1149728

 gain:[1] 0.2535969

 gain for fixed policy:[1] 0.2535969

 change in gain compared to fixed policy:[1] -4.529893e-11

RC: L , 

optimal policy:

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]    0    1    2    3    4    5    6    7    8     9     0

steady_prob:
          [,1]     [,2]       [,3]        [,4] 

In [ ]:
t = 2
r = 2
lam = 0.2763
mu = 0.2829
eps = 0.1718

costT = 1
mult = 0.32

costR = mult*costT*(mu+eps)

In [ ]:
aijMatrix = array(0, dim = c(t+1,t+1,r+1))

for (k in 1:(r+1)) {
  for (i in 1:t) {
    aijMatrix[i,i+1,k] <- lam
      if (i > 1) {

        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)
      }
    }
aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
aijMatrix[1,2,k] <- lam
}

for (k in 1:(r+1)) {
  for (i in 1:(t+1)) {

      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])
    }
  }

In [ ]:
print(aijMatrix)

, , 1

        [,1]    [,2]    [,3]
[1,] -0.2763  0.2763  0.0000
[2,]  0.1718 -0.4481  0.2763
[3,]  0.0000  0.3436 -0.3436

, , 2

        [,1]    [,2]    [,3]
[1,] -0.2763  0.2763  0.0000
[2,]  0.4547 -0.7310  0.2763
[3,]  0.0000  0.6265 -0.6265

, , 3

        [,1]    [,2]    [,3]
[1,] -0.2763  0.2763  0.0000
[2,]  0.4547 -0.7310  0.2763
[3,]  0.0000  0.9094 -0.9094



In [ ]:
policyOld = array(3, dim = c(1,t+1))
policyNew = array(1, dim=c(1,t+1))

In [ ]:
cat(policyOld,'\n\n')
cat(policyNew)

1 1 1 

1 2 1

In [ ]:
policyOld = policyNew
policy = policyNew

A = array(0, dim = c(t+1,t+1))
    for (i in 1:length(policy)) {
      A[i,] = aijMatrix[i,,(policy[i])]
    }
newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])

newA

1,0.2763,-0.2763
1,-0.4547,0.7310
1,0.0000,-0.3436


In [ ]:
q = array(0,dim=c(t+1,1))

for (i in 1:length(policy)) {

      q[i,1] = (policy[i]-1)*costR + (i-1)*eps*costT

}

q

0.000000
0.317304
0.343600


In [ ]:
solution = solve(newA) %*% q

 newsolution = array(0, dim=c(t+1,1))

    for (i in 1:t) {
      newsolution[i] = solution[i+1]
    }
newsolution[t+1] = 0

newsolution

-1.1219784
-0.4992179
0.0000000


In [ ]:
improveMatrix = array(0, dim=c(t+1,r+1))
cost = array(0,dim=c(t+1,r+1))

    for (i in 1:(t+1)) {
      for (j in 1:(r+1)) {
        # Cost when we have j rescue operation and i terrorists

      cost[i,j] = (j-1)*costR + (i-1)*eps*costT

      improveMatrix[i,j] = cost[i,j] + aijMatrix[i,,j] %*% newsolution

      }
    }

improveMatrix

0.1720687,0.3175727,0.4630767
0.2027437,0.1720687,0.3175727
0.1720687,0.1763440,0.1806192


In [ ]:
for (i in 1:length(policyNew)) {
      policyNew[i] = which.min(improveMatrix[i,])
    }

policyNew

1,2,1


In [ ]:
# CTMC MDP model for rescue operational system!!!

# Check function to determine new policy is same with current or not
checkPolicy = function(policySet, policySetnew) {
  n = 0
  for (i in policySet == policySetnew) {
    if (i == 'FALSE') {
      n = 1
    }
  }
  if (n == 0) {
    return("TRUE")
  } else {
    return("FALSE")
  }
}


# Define MDP function (for basic policy)
MDP = function(lam, mu, eps, numRescue, numTerror, costT, RC, mult) {
  r = numRescue
  t = numTerror

  #calculating costR

  costR = mult*costT*(mu+eps)


  #########################################################
  #EDITS: 22ND FEB,2023
  #NEW SERVICE RATES STRUCTURE, ADD ORIGINAL ONE AS WELL
  aijMatrix = array(0, dim = c(t+1,t+1,r+1))
  for (k in 1:(r+1)) {
    for (i in 1:t) {
      aijMatrix[i,i+1,k] <- lam
      if (i > 1) {
        #if (k==4){
          #if(i==4){
            #cat('min(k-1,i-1)',min(k-1,i-1))}}
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)
      }
    }
    aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
    aijMatrix[1,2,k] <- lam
}

  for (k in 1:(r+1)) {
    for (i in 1:(t+1)) {

      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])
    }
  }
  #########################################################


  # Design the MDP algorithm
  # Value determination
  policy = array(0, dim = c(1,t+1))
  policyOld = array(0, dim = c(1,t+1))
  policyNew = array(2, dim=c(1,t+1)) # initially set all decision is 1 in first policy
  policyTrack = array(2, dim=c(1,t+1))
  iteration = 0
  gainMatrix = c()

  while (checkPolicy(policyOld, policyNew) == 'FALSE') {
    policyOld = policyNew
    policy = policyNew
    # Design the transition matrix regarding policy
    A = array(0, dim = c(t+1,t+1))
    for (i in 1:length(policy)) {
      A[i,] = aijMatrix[i,,(policy[i])]
    }

    # Design the cost matrix regarding policy

    q = array(0,dim=c(t+1,1))
    for (i in 1:length(policy)) {

      if (RC == 'L'){
      cost = (policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost = sqrt(policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost = ((policy[i]-1)^2)*costR + (i-1)*eps*costT
      }

      q[i,1] = cost
    }


    newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
    solution = solve(newA) %*% q
    gain = solution[1] # First row of solution is gain value
    gainMatrix = append(gainMatrix, gain)

    # Policy improvement
    newsolution = array(0, dim=c(t+1,1))
    for (i in 1:t) {
      newsolution[i] = solution[i+1]
    }
    newsolution[t+1] = 0 # Set last v value as 0 to calcultate the test quantity in policy improvement
    improveMatrix = array(0, dim=c(t+1,r+1))
    cost = array(0,dim=c(t+1,r+1))

    for (i in 1:(t+1)) {
      for (j in 1:(r+1)) {
        # Cost when we have j rescue operation and i terrorists

        if (RC == 'L'){
      cost[i,j] = (j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost[i,j] = sqrt(j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost[i,j] = ((j-1)^2)*costR + (i-1)*eps*costT
      }

        improveMatrix[i,j] = cost[i,j] + aijMatrix[i,,j] %*% newsolution

      }
    }

    # pick the minimum value among the decision
    policyNew = array(1, dim = c(1,t+1))
    for (i in 1:length(policyNew)) {
      policyNew[i] = which.min(improveMatrix[i,])
    }
    iteration = iteration + 1
    policyTrack = rbind(policyTrack, policyNew)
    policyNew
  }
  for (i in 1:length(policyNew)) {
    policyNew[i] = policyNew[i] - 1
  }
  for (i in 1:nrow(policyTrack)) {
    for (j in 1:ncol(policyTrack)) {
      policyTrack[i,j] = policyTrack[i,j] - 1
    }

  }


   #return(list(policyNew,improveMatrix,cost,aijMatrix,newsolution))

   return (list(policyNew, improveMatrix, cost))

 }

In [ ]:
MDP(lam=0.2763, mu=0.2829, eps=0.1718, numRescue=10, numTerror=10, costT=1, RC='L', mult=0.32)

0,1,2,3,4,5,6,7,8,9,10
0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349,1.2113389,1.3568429,1.5023469,1.6478509
0.2447234,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349,1.2113389,1.3568429,1.5023469
0.2966359,0.2447234,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349,1.2113389,1.3568429
0.3485485,0.2966359,0.2447234,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349,1.2113389
0.4004610,0.3485484,0.2966359,0.2447234,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349
0.4523732,0.4004608,0.3485483,0.2966358,0.2447233,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309
0.5042829,0.4523709,0.4004589,0.3485469,0.2966349,0.2447229,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269
0.5561613,0.5042541,0.4523469,0.4004397,0.3485325,0.2966253,0.2447181,0.1928109,0.3383149,0.4838189,0.6293229
0.6076190,0.5557680,0.5039170,0.4520660,0.4002149,0.3483639,0.2965129,0.2446619,0.1928109,0.3383149,0.4838189
0.6527343,0.6016317,0.5505291,0.4994265,0.4483239,0.3972213,0.3461187,0.2950161,0.2439135,0.1928109,0.3383149


In [ ]:
imp

0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349,1.2113389,1.3568429,1.5023469,1.6478509
0.2447234,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349,1.2113389,1.3568429,1.5023469
0.2966359,0.2447234,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349,1.2113389,1.3568429
0.3485485,0.2966359,0.2447234,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349,1.2113389
0.4004610,0.3485484,0.2966359,0.2447234,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309,1.0658349
0.4523732,0.4004608,0.3485483,0.2966358,0.2447233,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269,0.9203309
0.5042829,0.4523709,0.4004589,0.3485469,0.2966349,0.2447229,0.1928109,0.3383149,0.4838189,0.6293229,0.7748269
0.5561613,0.5042541,0.4523469,0.4004397,0.3485325,0.2966253,0.2447181,0.1928109,0.3383149,0.4838189,0.6293229
0.6076190,0.5557680,0.5039170,0.4520660,0.4002149,0.3483639,0.2965129,0.2446619,0.1928109,0.3383149,0.4838189
0.6527343,0.6016317,0.5505291,0.4994265,0.4483239,0.3972213,0.3461187,0.2950161,0.2439135,0.1928109,0.3383149
0.5919754,0.5520590,0.5121425,0.4722261,0.4323096,0.3923931,0.3524767,0.3125602,0.2726438,0.2327273,0.1928109


In [ ]:
mult_test = function(cost_terror,rc,low_lim,up_lim,incr) {

prev_result = array(100,dim=c(1,11))

for (val in seq(low_lim,up_lim, by = incr)){

        current_result = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=2, numTerror=2, costT=cost_terror, RC=rc, mult=val)


        if (!(identical(current_result,prev_result))) {

          cat('\n\n','mult:',val,',','RC:',rc,',','\n\n')
          cat('optimal policy:\n')
          print(current_result)

          prev_result = current_result

        }

    }

}

In [ ]:
mult_test(1,'L',0,10,0.01)



 mult: 0 , RC: L , 

optimal policy:
     [,1] [,2] [,3]
[1,]    0    1    2


 mult: 0.32 , RC: L , 

optimal policy:
     [,1] [,2] [,3]
[1,]    0    1    0


 mult: 0.42 , RC: L , 

optimal policy:
     [,1] [,2] [,3]
[1,]    0    0    0
